In [1]:
import sys
sys.path.insert(1, '/home/genvekt/Dev/DL_Assignment_3/RU_EN_Translation/')
from collections import Counter

In [2]:
from data_loaders.vocabulary import Vocabulary
from data_loaders.batch import Batches
from utils import text_to_pickle, train_val_split
from torch import nn
from data_loaders.dataset import TextDataset
import torch 
import torch.nn.functional as F
import copy
import time
import math
from nltk.tokenize import word_tokenize
import pickle
from tqdm import tqdm

[nltk_data] Downloading package punkt to /home/genvekt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Take first 200 000 sentences in the dataset

In [3]:
def get_subset(input_file, output_file, amount):
    with open(output_file, "w") as out_f:
        with open (input_file, "r") as f:
            for line in f:
                out_f.write(line)
                amount -= 1
                if amount <=0:
                    break

In [8]:
get_subset("../data/corpus.en_ru.1m.ru", "../data/corpus.en_ru.small.ru", 200000)

In [9]:
get_subset("../data/corpus.en_ru.1m.en", "../data/corpus.en_ru.small.en", 200000)

In [10]:
text_to_pickle("../data/corpus.en_ru.small.en", "../data/corpus.en_ru.small.en_tokens.p", language="english")
text_to_pickle("../data/corpus.en_ru.small.ru", "../data/corpus.en_ru.small.ru_tokens.p", language="russian")

100%|██████████| 200000/200000 [00:30<00:00, 6602.04it/s]


## Parse Russian Vocabulary

In [12]:
vocab = Vocabulary()
vocab.parse_file("../data/corpus.en_ru.small.ru", language='russian')
vocab.dump("../data/vocabularies/ru_small.voc")

100%|██████████| 200000/200000 [00:31<00:00, 6340.87it/s]


## Parse English Vocablary

In [13]:
vocab = Vocabulary()
vocab.parse_file("../data/corpus.en_ru.small.en", language='english')
vocab.dump("../data/vocabularies/en_small.voc")

100%|██████████| 200000/200000 [00:26<00:00, 7434.39it/s]


# Parse texts to tokens

In [3]:
text_to_pickle("../data/corpus.en_ru.1m.en", "../data/corpus.en_ru.1m.en_tokens.p", language="english")

100%|██████████| 1000000/1000000 [02:15<00:00, 7356.39it/s]


In [4]:
text_to_pickle("../data/corpus.en_ru.1m.ru", "../data/corpus.en_ru.1m.ru_tokens.p", language="russian")

100%|██████████| 1000000/1000000 [02:35<00:00, 6415.40it/s]


# Create bigram matrice for target vocabulary, will use it in test

In [102]:
full_ds = TextDataset(ru_tokens="../data/corpus.en_ru.small.ru_tokens.p",
                 ru_voc="../data/vocabularies/ru_small.voc",
                 en_tokens="../data/corpus.en_ru.small.en_tokens.p",
                 en_voc="../data/vocabularies/en_small.voc", 
                 random_sub=0.00)

In [109]:
from tqdm  import tqdm

In [120]:
EN_V = len(full_ds.en_voc)
bigram = {}
for sent in tqdm(full_ds.en):
    full_sent = sent + ["<EOS>"]
    for i in range(len(full_sent)-1):
        t = full_sent[i] 
        t_next = full_sent[i+1]
        t_id = full_ds.en_voc.word_2_idx[t]
        t_next_id = full_ds.en_voc.word_2_idx[t_next]
        if not t_id in bigram:
            bigram[t_id] = {}
        if not t_next_id in bigram[t_id]:
            bigram[t_id][t_next_id] = 1
        else:
            bigram[t_id][t_next_id] += 1

100%|██████████| 200000/200000 [00:03<00:00, 50877.15it/s]


# Convert frequences into probabilities

In [121]:
for t1 in bigram:
    total_sum = 0
    for t2 in bigram[t1]:
        total_sum += bigram[t1][t2]
    for t2 in bigram[t1]:
        bigram[t1][t2] /= total_sum

# Save bigram

In [127]:
with open("../data/en_biram_small.p", "wb") as f:
    pickle.dump(bigram, f)

# Split corpus into 80% train and 20% test splits

In [11]:
train_val_split(text_file1="../data/corpus.en_ru.small.ru_tokens.p",
                text_file2="../data/corpus.en_ru.small.en_tokens.p",
                train_file1="../data/corpus.en_ru.small.ru_tokens_train.p",
                train_file2="../data/corpus.en_ru.small.en_tokens_train.p",
                val_file1="../data/corpus.en_ru.small.ru_tokens_val.p",
                val_file2="../data/corpus.en_ru.small.en_tokens_val.p")

# Train transformer

In [3]:
train_ds = TextDataset(ru_tokens="../data/corpus.en_ru.small.ru_tokens_train.p",
                 ru_voc="../data/vocabularies/ru_small.voc",
                 en_tokens="../data/corpus.en_ru.small.en_tokens_train.p",
                 en_voc="../data/vocabularies/en_small.voc", 
                 random_sub=0.01)

In [4]:
val_ds = TextDataset(ru_tokens="../data/corpus.en_ru.small.ru_tokens_val.p",
                 ru_voc="../data/vocabularies/ru_small.voc",
                 en_tokens="../data/corpus.en_ru.small.en_tokens_val.p",
                 en_voc="../data/vocabularies/en_small.voc", 
                 random_sub=0.0)

In [5]:
batched_train_ds = Batches(train_ds, 16, train=True)

In [12]:
batched_val_ds = Batches(train_ds, 16, train=True)

In [7]:
import numpy as np
from torch.autograd import Variable

def nopeak_mask(size):
    np_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    return np_mask

def get_masks(src_batch, src_pad, trg_batch=None, trg_pad=None):
    src_msk = (src_batch != src_pad).unsqueeze(-2).long()
    
    if trg_batch is not None:
        trg_msk = (trg_batch != trg_pad).unsqueeze(-2).bool()
        trg_len = trg_batch.size(1)
        np_msk = nopeak_mask(trg_len).type_as(trg_batch).bool()
        trg_msk = (trg_msk & np_msk).long()
    else:
        trg_mask = None
    return src_msk, trg_msk

In [8]:
def importance_sampling(scores, top_m=10000, rand_n=10000):
    """
    Computes sampled softmax function
    """
    V = scores.shape[-1]       #  The target vocabular size 
    B = scores.shape[0]        # The batch size
    
    # Take top M words
    x_m = scores[:, :,:top_m]
    
    # Randomly select N words from other V-M words
    ids = (torch.randperm(V-top_m)+top_m)[:rand_n]
    x_n = scores[:,:,ids]
    
    # Compute approximation of softmax
    alpha = (V - top_m)/rand_n
    Z = torch.sum(torch.exp(x_m),2) + alpha*torch.sum(torch.exp(x_n),2)
    return torch.log(torch.exp(scores)/Z.unsqueeze(2).repeat(1,1,V))

In [9]:
from model.model import Transformer

In [10]:
d_model = 256
heads = 8
N = 4
src_vocab = len(train_ds.ru_voc)
trg_vocab = len(train_ds.en_voc)

model = Transformer(src_vocab, trg_vocab, d_model, N, heads)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
        
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [26]:
model.train()

total_loss = 0
criterion = nn.NLLLoss(ignore_index=batched_train_ds.pad_idx(is_source=False))
for epoch in range(1):
    for i, batch in enumerate(batched_train_ds.iterate()):
        src, trg = batch

        trg_input = trg[:, :-1]
        targets = trg[:, 1:].contiguous().view(-1)

        src_msk, trg_msk = get_masks(src, batched_train_ds.pad_idx(is_source=True),
                                     trg_input, batched_train_ds.pad_idx(is_source=False))
        
        preds = model(src, trg_input, src_msk, trg_msk)
        preds = importance_sampling(preds)

        optim.zero_grad()

        loss = criterion(preds.view(-1, preds.shape[-1]), targets)
        
        loss.backward()
        optim.step()
        total_loss += loss
        
        if (i + 1) % 10 == 0:
            loss_avg = total_loss / 10
            print("epoch %d, iter = %d, loss = %.4f" % (epoch + 1, i + 1, loss_avg))
            total_loss = 0
            
        if i > 300:
            break
    
    # Validation
    for i, batch in enumerate(batched_val_ds.iterate()):
        src, trg = batch

        trg_input = trg[:, :-1]
        targets = trg[:, 1:].contiguous().view(-1)

        src_msk, trg_msk = get_masks(src, batched_train_ds.pad_idx(is_source=True),
                                        trg_input, batched_train_ds.pad_idx(is_source=False))
        
        with torch.no_grad():
            preds = model(src, trg_input, src_msk, trg_msk)
        
        preds = importance_sampling(preds)
        loss = criterion(preds.view(-1, preds.shape[-1]), targets)
        
        total_loss += loss
        if i > 100:
            print("Val loss = %.3f" % (total_loss/100))
            total_loss = 0
            break

epoch 1, iter = 10, loss = 4.6770
epoch 1, iter = 20, loss = 4.6450
epoch 1, iter = 30, loss = 4.5972
epoch 1, iter = 40, loss = 4.6452
epoch 1, iter = 50, loss = 4.6282
epoch 1, iter = 60, loss = 4.7003
epoch 1, iter = 70, loss = 4.5950
epoch 1, iter = 80, loss = 4.6205
epoch 1, iter = 90, loss = 4.6056
epoch 1, iter = 100, loss = 4.6566
epoch 1, iter = 110, loss = 4.5939
epoch 1, iter = 120, loss = 4.5942
epoch 1, iter = 130, loss = 4.6553
epoch 1, iter = 140, loss = 4.5543
epoch 1, iter = 150, loss = 4.7331
epoch 1, iter = 160, loss = 4.5873
epoch 1, iter = 170, loss = 4.7067
epoch 1, iter = 180, loss = 4.6487
epoch 1, iter = 190, loss = 4.5904
epoch 1, iter = 200, loss = 4.6456
epoch 1, iter = 210, loss = 4.6629
epoch 1, iter = 220, loss = 4.5853
epoch 1, iter = 230, loss = 4.6667
epoch 1, iter = 240, loss = 4.5974
epoch 1, iter = 250, loss = 4.5265
epoch 1, iter = 260, loss = 4.6458
epoch 1, iter = 270, loss = 4.5425
epoch 1, iter = 280, loss = 4.5309
epoch 1, iter = 290, loss = 4

In [27]:
torch.save(model, "model_small.ckpt")

# Test transformer

In [18]:
with open("../data/en_biram_small.p", 'rb') as f:
    bigram = pickle.load(f)

In [35]:
def translate(m, sent, src_voc, trg_voc, bigram):
    beam_window = 3
    m.eval()
    tokens = word_tokenize(sent, language='russian')
    print(tokens)
    src = []
    for t in tokens:
        if t in src_voc.word_2_idx:
            src.append(src_voc.word_2_idx[t])
        else:
            src.append(src_voc.word_2_idx["<UNK>"])
    src = torch.tensor([src]).long()
    src_mask = (src != src_voc.word_2_idx["<PAD>"]).unsqueeze(-2)
    
    e_outputs = m.encoder(src, src_mask)
    
    outputs = torch.zeros(100).type_as(src)
    outputs[0] = trg_voc.word_2_idx["<SOS>"]
    
    for i in range(1, 100):
        trg_mask = np.triu(np.ones((1, i, i)), k=1).astype('uint8')
        trg_mask = torch.from_numpy(trg_mask) == 0
        out = m.out_fc(m.decoder(outputs[:i].unsqueeze(0), e_outputs, src_mask, trg_mask))
        out = importance_sampling(out)  
        
        best_seq = beam_search(out, bigram=None, beam_window=beam_window)
                           
        outputs[i] = best_seq[0,-1]
        if outputs[i] == trg_voc.word_2_idx['<EOS>']:
            break
    return ' '.join([trg_voc.idx_2_word[int(ix)] for ix in outputs[1:i]])

In [36]:
translate(model, 
         "Отдел полевого персонала с тех пор анализирует данные в разбивке по областям деятельности, уровню должностей и миссиям в стремлении выявить возможности для улучшения.",
         train_ds.ru_voc,
         train_ds.en_voc, bigram=None)

['Отдел', 'полевого', 'персонала', 'с', 'тех', 'пор', 'анализирует', 'данные', 'в', 'разбивке', 'по', 'областям', 'деятельности', ',', 'уровню', 'должностей', 'и', 'миссиям', 'в', 'стремлении', 'выявить', 'возможности', 'для', 'улучшения', '.']


'In the , the , the the , the the , , , the the the the be , , , , , , , , , , the the the the the the the the the the the the the the the the the the be , , , , , , , , , , , , , , , , , , the the .'

In [31]:
translate(model, 
         "Отдел полевого персонала с тех пор анализирует данные в разбивке по областям деятельности, уровню должностей и миссиям в стремлении выявить возможности для улучшения.",
         train_ds.ru_voc,
         train_ds.en_voc, bigram)

['Отдел', 'полевого', 'персонала', 'с', 'тех', 'пор', 'анализирует', 'данные', 'в', 'разбивке', 'по', 'областям', 'деятельности', ',', 'уровню', 'должностей', 'и', 'миссиям', 'в', 'стремлении', 'выявить', 'возможности', 'для', 'улучшения', '.']
[8975, 41112, 1242, 9, 150, 381, 29419, 311, 7, 26927, 13, 19893, 156, 4, 3703, 8780, 6, 72974, 7, 7225, 7464, 217, 12, 1231, 5]


'In the , the , the the , the the , , , the the the the be , , , , , , , , , , the the the the the the the the the the the the the the the the the the be , , , , , , , , , , , , , , , , , , the the .'

In [21]:
def beam_search(in_scores, bigram=None, beam_window=2):
    # -> [B, S, V]
    B = in_scores.shape[0]
    S = in_scores.shape[1]
    V = in_scores.shape[2]
    
    res = torch.zeros((B, S), dtype=torch.long)
    
    cand_vs, cand_ids = in_scores.topk(beam_window, dim=-1)
    for b_idx in range(B):
        best_seqs = []
        for word_idx in range(S):
            # Get best candidates for given position
            cand_idx = cand_ids[b_idx, word_idx]
            cand_v = cand_vs[b_idx, word_idx]
            # Calculate all possible outputs
            seqs = []
            for cand_pos in range(len(cand_v)):
                if len(best_seqs) == 0:
                    seqs.append([cand_idx[cand_pos].unsqueeze(0),cand_v[cand_pos]])
                else:
                    for best_seq in best_seqs:
                        last_word = best_seq[0][-1]
                        cand_bigram_prob = -100
                        if bigram is not None and (last_word in bigram and cand_idx[cand_pos] in bigram[last_word]):
                            cand_bigram_prob = torch.log(torch.tensor(bigram[last_word][cand_idx[cand_pos]]))
                        
                        new_seq = torch.cat([best_seq[0], cand_idx[cand_pos].unsqueeze(0)])
                        new_score = best_seq[1] - cand_v[cand_pos] - cand_bigram_prob
                        seqs.append([new_seq, new_score])
            best_seqs = sorted(seqs, key=lambda x: x[1])[:beam_window]
        res[b_idx] = best_seqs[0][0]
    return res

In [37]:
with open("../data/eval-ru-100.txt", 'r') as f:
    test_lines = list(f.readlines())
    
with open("answer.txt", 'w') as f:
    for line in tqdm(test_lines):
        answ = translate(model, 
             line,
             train_ds.ru_voc,
             train_ds.en_voc, 
             bigram)
        f.write(answ+"\n")

  0%|          | 0/100 [00:00<?, ?it/s]

['26', '.', 'Вопрос', 'о', 'лесах', 'необходимо', 'вывести', 'на', 'более', 'высокий', 'уровень', 'в', 'рамках', 'целей', 'устойчивого', 'развития', ',', 'в', 'том', 'числе', 'посредством', 'включения', 'в', 'такие', 'цели', 'убедительных', 'и', 'четких', 'целевых', 'и', 'рабочих', 'показателей', 'по', 'лесам', '.']


  1%|          | 1/100 [00:06<11:19,  6.86s/it]

['В', 'рамках', 'экологической', 'экспертизы', 'определены', 'пять', 'вариантов', 'строительства', 'и', 'эксплуатации', 'замещающей', 'электростанции', ',', 'которая', 'восстановит', 'мощность', 'энергораспределительной', 'сети', 'Управления', 'по', 'состоянию', 'до', 'стихийного', 'бедствия', '.']


  2%|▏         | 2/100 [00:08<06:38,  4.07s/it]

...

 98%|█████████▊| 98/100 [11:59<00:15,  7.58s/it]

['Это', 'выступление', 'является', 'безответственным', ',', 'так', 'как', 'оно', 'непосредственно', 'бросает', 'вызов', 'основным', 'принципам', 'права', 'вооруженных', 'конфликтов', 'и', 'нормам', 'международного', 'гуманитарного', 'права', 'и', 'подрывает', 'международный', 'мир', 'и', 'безопасность', '.']


 99%|█████████▉| 99/100 [12:07<00:07,  7.51s/it]

['14', '.', 'Женевская', 'Конференция', 'по', 'разоружению', 'является', 'единственным', 'надлежащим', 'форумом', 'для', 'ведения', 'переговоров', 'относительно', 'договора', 'о', 'запрещении', 'производства', 'расщепляющего', 'материала', 'для', 'ядерного', 'оружия', 'или', 'других', 'ядерных', 'взрывных', 'устройств', '.']


100%|██████████| 100/100 [12:15<00:00,  7.36s/it]
